### Imports

In [1]:
import pandas as pd
import numpy as np
import openpyxl

### Load Data

In [2]:
df = pd.read_excel(r"data_masterstudie_2022-07-25_09-26.xlsx")

In [3]:
df

,CASE,SERIAL,REF,QUESTNNR,MODE,STARTED,AE01_01,AE01_02,AE02_01,AE03_01,...,MAILSENT,LASTDATA,FINISHED,Q_VIEWER,LASTPAGE,MAXPAGE,MISSING,MISSREL,TIME_RSI,DEG_TIME
0,Interview-Nummer (fortlaufend),Seriennummer (sofern verwendet),Referenz (sofern im Link angegeben),"Fragebogen, der im Interview verwendet wurde",Interview-Modus,Zeitpunkt zu dem das Interview begonnen hat (E...,Rsikoverhalten: Investition in Alternative A,Rsikoverhalten: Investition in Alternative B,Quelle: Erbe,ErbeRiskant: Alternative 2,...,Versandzeitpunkt der Einladungsmail (nur für n...,Zeitpunkt als der Datensatz das letzte mal geä...,Wurde die Befragung abgeschlossen (letzte Seit...,Hat der Teilnehmer den Fragebogen nur angesehe...,"Seite, die der Teilnehmer zuletzt bearbeitet hat","Letzte Seite, die im Fragebogen bearbeitet wurde",Anteil fehlender Antworten in Prozent,Anteil fehlender Antworten (gewichtet nach Rel...,Maluspunkte für schnelles Ausfüllen,Maluspunkte für schnelles Ausfüllen
1,250,NaN,NaN,base,interview,2022-07-06 09:52:30,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 09:56:06,1,0,7,7,3,0,1.83,43
2,253,NaN,NaN,base,interview,2022-07-06 13:27:25,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 13:35:37,1,0,7,7,6,3,0.84,1
3,254,NaN,NaN,base,interview,2022-07-06 13:30:33,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 13:35:32,1,0,7,7,3,0,1.46,23
4,255,NaN,NaN,base,interview,2022-07-06 14:19:42,NaN,NaN,NaN,NaN,...,NaN,2022-07-06 14:33:27,1,0,7,7,71,73,1.26,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,537,NaN,NaN,base,interview,2022-07-14 13:11:05,NaN,NaN,NaN,NaN,...,NaN,2022-07-14 13:15:24,1,0,7,7,3,0,1.82,55
188,538,NaN,NaN,base,interview,2022-07-14 13:16:04,NaN,NaN,NaN,NaN,...,NaN,2022-07-14 13:27:32,1,0,7,7,3,0,1.65,73
189,539,NaN,NaN,base,interview,2022-07-14 13:29:02,NaN,NaN,NaN,NaN,...,NaN,2022-07-14 13:38:59,1,0,7,7,3,0,1.16,31
190,540,NaN,NaN,base,interview,2022-07-14 19:49:45,NaN,NaN,NaN,NaN,...,NaN,2022-07-14 20:07:49,1,0,7,7,3,0,0.41,0


### Filter for relevant columns

In [4]:
df = df[["FS01_CP","FS01","SD03","SD05","SD11","SD11_10","SD14","SD14_08","SD16","MZ03_01","MZ03_02","MZ03_03","MZ03_04","MZ03_05","MZ03_06","SC01","SC02","VR01_01",
"AN01_01","AN01_02","AN02_01","AN02_02","AN03_01","AN03_02","AN04_01","AN04_02","RI01","RI02","RI03","RI04","RI05","RI06","RI07","RI08","RI09","RI10","RI11","RI12","RI13","TIME_SUM","FINISHED"
]]

In [5]:
df.columns

Index(['FS01_CP', 'FS01', 'SD03', 'SD05', 'SD11', 'SD11_10', 'SD14', 'SD14_08',
       'SD16', 'MZ03_01', 'MZ03_02', 'MZ03_03', 'MZ03_04', 'MZ03_05',
       'MZ03_06', 'SC01', 'SC02', 'VR01_01', 'AN01_01', 'AN01_02', 'AN02_01',
       'AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02', 'RI01', 'RI02',
       'RI03', 'RI04', 'RI05', 'RI06', 'RI07', 'RI08', 'RI09', 'RI10', 'RI11',
       'RI12', 'RI13', 'TIME_SUM', 'FINISHED'],
      dtype='object')

### Drop first row 

In [6]:
df = df.iloc[1: , :]

### Subtract 1 from some columns because they start at 1 not at 0
AN01_01 and AN01_02 must sum up to 100

In [7]:
df[['AN01_01','AN01_02','AN02_01','AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02']] = df[['AN01_01','AN01_02','AN02_01','AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02']] -1

In [8]:
df[['AN01_01','AN01_02','AN02_01','AN02_02', 'AN03_01', 'AN03_02', 'AN04_01', 'AN04_02']]

,AN01_01,AN01_02,AN02_01,AN02_02,AN03_01,AN03_02,AN04_01,AN04_02
1,NaN,NaN,8,92,52,48,82,18
2,NaN,NaN,20,80,20,80,20,80
3,28,72,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,-10,-10,-10,-10,-10,-10
5,50,50,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
187,NaN,NaN,25,75,18,82,70,30
188,20,80,NaN,NaN,NaN,NaN,NaN,NaN
189,NaN,NaN,16,84,19,81,72,28
190,NaN,NaN,56,44,80,20,86,14


### Exclude data with short answer time (<3 minutes) and data where no consent/the last page was not reached or a lot of questions were not answered.

In [9]:
len(df.index)

191

In [10]:
df_short_answer = df[df["TIME_SUM"] >= 180 ]

In [11]:
len(df_short_answer.index)

164

In [12]:
df_finished = df_short_answer[df_short_answer["FINISHED"] == 1] 

In [13]:
len(df_finished.index)

164

In [14]:
df_consent = df_finished[df_finished["SC01"] == 2 ]

In [15]:
len(df_consent.index)

164

In [16]:
df_overall = df_consent[df_consent["SC02"] == 1]

In [17]:
len(df_overall.index)

163

In [18]:
df_final = df_overall.loc[(df_overall["RI01"] != -9)  & (df_overall["RI02"] != -9)]

In [19]:
len(df_final.index)

162

In [20]:
df_final = df_final.loc[(df_final["AN01_01"] != -10)  & (df_final["AN01_02"] != -10) &(df_final["AN02_01"] != -10) &(df_final["AN02_02"] != -10) &(df_final["AN03_01"] != -10) &(df_final["AN03_02"] != -10) &(df_final["AN04_01"] != -10) &(df_final["AN04_02"] != -10)]

In [21]:
len(df_final.index)

161

In [22]:
df_final = df_final.reset_index(drop=True)

### Create a Column with the total amount invested in the risky/save alternative

In [23]:

df_final['SAVE_TOTAL']= df_final['AN02_02'].where(df_final['FS01'] ==1, 0) * 0.2 + df_final['AN03_02'].where(df_final['FS01'] ==1, 0) * 0.7 + df_final['AN04_02'].where(df_final['FS01'] ==1, 0) * 0.1 + df_final['AN02_02'].where(df_final['FS01'] ==2, 0) * 0.2 + df_final['AN03_02'].where(df_final['FS01'] ==2, 0) * 0.5 + df_final['AN04_02'].where(df_final['FS01'] ==2, 0) * 0.3  + df_final['AN01_02'].where(df_final['FS01'] ==3, 0)

df_final['RISK_TOTAL']= df_final['AN02_01'].where(df_final['FS01'] ==1, 0) * 0.2 + df_final['AN03_01'].where(df_final['FS01'] ==1, 0) * 0.7 + df_final['AN04_01'].where(df_final['FS01'] ==1, 0) * 0.1 + df_final['AN02_01'].where(df_final['FS01'] ==2, 0) * 0.2 + df_final['AN03_01'].where(df_final['FS01'] ==2, 0) * 0.5 + df_final['AN04_01'].where(df_final['FS01'] ==2, 0) * 0.3 + df_final['AN01_01'].where(df_final['FS01'] ==3, 0)

#### SAVE_TOTAL and RISK_TOTAL have to sum up to 100

In [24]:
df_final

,FS01_CP,FS01,SD03,SD05,SD11,SD11_10,SD14,SD14_08,SD16,MZ03_01,...,RI08,RI09,RI10,RI11,RI12,RI13,TIME_SUM,FINISHED,SAVE_TOTAL,RISK_TOTAL
0,0,1,4,2,8,NaN,3,NaN,7,4,...,2,1,2,1,1,2,215,1,53.8,46.2
1,0,2,11,2,8,NaN,6,NaN,11,1,...,3,1,2,2,2,1,492,1,80.0,20.0
2,0,3,3,1,7,NaN,3,NaN,4,5,...,2,1,2,1,1,1,299,1,72.0,28.0
3,1,3,3,1,8,NaN,3,NaN,5,4,...,3,2,2,4,1,3,391,1,50.0,50.0
4,1,2,5,2,8,NaN,4,NaN,6,2,...,1,1,1,3,1,2,566,1,50.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,61,2,4,2,8,NaN,4,NaN,7,4,...,2,1,2,2,2,2,232,1,65.0,35.0
157,61,3,7,1,8,NaN,4,NaN,8,4,...,1,1,2,1,1,1,332,1,80.0,20.0
158,61,1,4,2,7,NaN,4,NaN,7,4,...,2,1,2,2,2,2,302,1,76.3,23.7
159,62,2,8,1,4,NaN,4,NaN,5,5,...,3,2,2,1,1,1,382,1,23.0,77.0


### Only include cases where max 1 item of the 13-Item Risk Tolerance Scale is missing

In [25]:
df_final = df_final[(df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']] == -9).sum(axis=1) < 2]

In [26]:
df_final = df_final.replace(-9, np.NaN)

### The 13-Item Risk Tolerance Scale has reversed item number on some items, I was not able to do in the Study 

Scoring
1. A = 4 B = 3 C = 2 D = 1
2. A = 1 B = 2 C = 3 D = 4
3. A = 1 B = 2 C = 3 D = 4
4. A = 1 B = 2 C = 3
5. A = 1 B = 2 C = 3
6. A = 1 B = 2 C = 3 D = 4
7. A = 1 B = 2 C = 3 D = 4
8. A = 1 B = 2 C = 3 D = 4
9. A = 1 B = 3*
10. A = 1 B = 3
11. A = 1 B = 2 C = 3 D = 4
12. A = 1 B = 2 C = 3 D = 4
13. A = 1 B = 2 C = 3 D = 4

In [27]:
df_final['RI01'] = df_final['RI01'] + 4

In [28]:
df_final['RI01']= np.where(df_final['RI01'] == 5, 4, df_final.RI01)
df_final['RI01']=np.where(df_final['RI01'] == 6, 3, df_final.RI01)
df_final['RI01']=np.where(df_final['RI01'] == 7, 2, df_final.RI01)
df_final['RI01']=np.where(df_final['RI01'] == 8, 1, df_final.RI01)

In [29]:
df_final['RI09']= np.where(df_final['RI09'] == 2, 3, df_final.RI09)
df_final['RI10']= np.where(df_final['RI10'] == 2, 3, df_final.RI10)

#### Replace missing values of the 13-Item Risk Tolerance Scale with average (There are only very few)

In [30]:
df_final.loc[df_final['RI01'].isna(), 'RI01'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI02'].isna(), 'RI02'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI03'].isna(), 'RI03'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI04'].isna(), 'RI04'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI05'].isna(), 'RI05'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI06'].isna(), 'RI06'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI07'].isna(), 'RI07'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI08'].isna(), 'RI08'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI09'].isna(), 'RI09'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI10'].isna(), 'RI10'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI11'].isna(), 'RI11'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI12'].isna(), 'RI12'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()
df_final.loc[df_final['RI13'].isna(), 'RI13'] = df_final[['RI01','RI02','RI03','RI04','RI05','RI06','RI07','RI08','RI09','RI10','RI11','RI12','RI13']].mean(axis=1).round()

### Create a Column with the total sum of the 13-Item Risk Tolerance Scale

In [31]:
df_final["RISK_TOLERANCE_TOTAL"] = df_final['RI01'] + df_final['RI02'] +df_final['RI03'] +df_final['RI04'] +df_final['RI05'] +df_final['RI06'] +df_final['RI07'] +df_final['RI08'] +df_final['RI09']+ df_final['RI10'] +df_final['RI11']+df_final['RI12']+df_final['RI13']

33 – 47 High tolerance for risk <br>
29 – 32 Above-average tolerance for risk <br>
23 – 28 Average/moderate tolerance for risk <br> 
19 – 22 Below-average tolerance for risk <br>
0 – 18 Low tolerance for risk <br>

In [32]:

conditions = [
    (df_final['RISK_TOLERANCE_TOTAL'] >= 33) & (df_final['RISK_TOLERANCE_TOTAL'] <= 47),
 (df_final['RISK_TOLERANCE_TOTAL'] >= 29) & (df_final['RISK_TOLERANCE_TOTAL'] <= 32),
 (df_final['RISK_TOLERANCE_TOTAL'] >= 23) & (df_final['RISK_TOLERANCE_TOTAL'] <= 28),
  (df_final['RISK_TOLERANCE_TOTAL'] >= 19) & (df_final['RISK_TOLERANCE_TOTAL'] <= 22),
   (df_final['RISK_TOLERANCE_TOTAL'] >= 0) & (df_final['RISK_TOLERANCE_TOTAL'] <= 18)
    ]


values = ['High tolerance for risk', 'Above-average tolerance for risk', 'Average/moderate tolerance for risk', 'Below-average tolerance for risk', 'Low tolerance for risk']


df_final["RISK_RESULT"] = np.select(conditions, values)

### Split by groups

In [ ]:
df_final['RISK_LOTTO_GRUPPE_A_NORM'] =  df_final['AN04_01'].where(df_final['FS01'] ==1, np.NaN) 
df_final['RISK_LOTTO_GRUPPE_B_NORM'] =  df_final['AN04_01'].where(df_final['FS01'] ==2, np.NaN)  
df_final['SAFE_LOTTO_GRUPPE_A_NORM'] = df_final['AN04_02'].where(df_final['FS01'] ==1, np.NaN) 
df_final['SAFE_LOTTO_GRUPPE_B_NORM'] = df_final['AN04_02'].where(df_final['FS01'] ==2, np.NaN) 
df_final['RISK_ERSPART_GRUPPE_A_NORM'] = df_final['AN02_01'].where(df_final['FS01'] ==1, np.NaN) 
df_final['RISK_ERSPART_GRUPPE_B_NORM'] = df_final['AN02_01'].where(df_final['FS01'] ==2, np.NaN) 
df_final['SAFE_ERSPART_GRUPPE_A_NORM']= df_final['AN02_02'].where(df_final['FS01'] ==1, np.NaN)
df_final['SAFE_ERSPART_GRUPPE_B_NORM'] = df_final['AN02_02'].where(df_final['FS01'] ==2, np.NaN) 
df_final['RISK_ERBE_GRUPPE_A_NORM']= df_final['AN03_01'].where(df_final['FS01'] ==1, np.NaN)  
df_final['RISK_ERBE_GRUPPE_B_NORM'] = df_final['AN03_01'].where(df_final['FS01'] ==2, np.NaN)
df_final['SAFE_ERBE_GRUPPE_A_NORM']= df_final['AN03_02'].where(df_final['FS01'] ==1, np.NaN) 
df_final['SAFE_ERBE_GRUPPE_B_NORM'] = df_final['AN03_02'].where(df_final['FS01'] ==2, np.NaN) 
df_final['RISK_KONTROLLE_NORM'] = df_final['AN01_01'].where(df_final['FS01'] ==3, np.NaN) 
df_final['SAFE_KONTROLLE_NORM'] = df_final['AN01_02'].where(df_final['FS01'] ==3, np.NaN) 

### Create absolute values of how much was invested in risky/safe Alternative, right now there are percentages

In [33]:
df_final['RISK_LOTTO_GRUPPE_A'] =  df_final['AN04_01'].where(df_final['FS01'] ==1, np.NaN) * 100
df_final['RISK_LOTTO_GRUPPE_B'] =  df_final['AN04_01'].where(df_final['FS01'] ==2, np.NaN) * 300  
df_final['SAFE_LOTTO_GRUPPE_A'] = df_final['AN04_02'].where(df_final['FS01'] ==1, np.NaN) * 100 
df_final['SAFE_LOTTO_GRUPPE_B'] = df_final['AN04_02'].where(df_final['FS01'] ==2, np.NaN) * 300
df_final['RISK_ERSPART_GRUPPE_A'] = df_final['AN02_01'].where(df_final['FS01'] ==1, np.NaN) * 200  
df_final['RISK_ERSPART_GRUPPE_B'] = df_final['AN02_01'].where(df_final['FS01'] ==2, np.NaN) * 200
df_final['SAFE_ERSPART_GRUPPE_A']= df_final['AN02_02'].where(df_final['FS01'] ==1, np.NaN) * 200  
df_final['SAFE_ERSPART_GRUPPE_B'] = df_final['AN02_02'].where(df_final['FS01'] ==2, np.NaN) * 200
df_final['RISK_ERBE_GRUPPE_A']= df_final['AN03_01'].where(df_final['FS01'] ==1, np.NaN) * 700  
df_final['RISK_ERBE_GRUPPE_B'] = df_final['AN03_01'].where(df_final['FS01'] ==2, np.NaN) * 500
df_final['SAFE_ERBE_GRUPPE_A']= df_final['AN03_02'].where(df_final['FS01'] ==1, np.NaN) * 700  
df_final['SAFE_ERBE_GRUPPE_B'] = df_final['AN03_02'].where(df_final['FS01'] ==2, np.NaN) * 500
df_final['RISK_KONTROLLE'] = df_final['AN01_01'].where(df_final['FS01'] ==3, np.NaN) * 1000
df_final['SAFE_KONTROLLE'] = df_final['AN01_02'].where(df_final['FS01'] ==3, np.NaN) * 1000

### Create the sum of money invested in risky/safe Alternative

In [34]:
df_final['TOTAL_RISIKO'] = df_final.fillna(0)['RISK_LOTTO_GRUPPE_A'] + df_final.fillna(0)['RISK_LOTTO_GRUPPE_B'] + df_final.fillna(0)['RISK_ERSPART_GRUPPE_A'] + df_final.fillna(0)['RISK_ERSPART_GRUPPE_B'] + df_final.fillna(0)['RISK_ERBE_GRUPPE_A'] + df_final.fillna(0)['RISK_ERBE_GRUPPE_B'] + df_final.fillna(0)['RISK_KONTROLLE']
df_final['TOTAL_SAFE'] = df_final.fillna(0)['SAFE_LOTTO_GRUPPE_A'] + df_final.fillna(0)['SAFE_LOTTO_GRUPPE_B'] + df_final.fillna(0)['SAFE_ERSPART_GRUPPE_A'] + df_final.fillna(0)['SAFE_ERSPART_GRUPPE_B'] + df_final.fillna(0)['SAFE_ERBE_GRUPPE_A'] + df_final.fillna(0)['SAFE_ERBE_GRUPPE_B'] + df_final.fillna(0)['SAFE_KONTROLLE']

In [35]:
# df_final['TOTAL_RISIKO'] = df_final['RISK_LOTTO'] + df_final['RISK_ERSPART'] + df_final['RISK_ERBE'] +df_final['RISK_KONTROLLE']
# df_final['TOTAL_SAFE'] = df_final['SAFE_LOTTO'] + df_final['SAFE_ERSPART'] + df_final['SAFE_ERBE']+ df_final['SAFE_KONTROLLE']

In [36]:
df_final

,FS01_CP,FS01,SD03,SD05,SD11,SD11_10,SD14,SD14_08,SD16,MZ03_01,...,SAFE_ERSPART_GRUPPE_A,SAFE_ERSPART_GRUPPE_B,RISK_ERBE_GRUPPE_A,RISK_ERBE_GRUPPE_B,SAFE_ERBE_GRUPPE_A,SAFE_ERBE_GRUPPE_B,RISK_KONTROLLE,SAFE_KONTROLLE,TOTAL_RISIKO,TOTAL_SAFE
0,0,1,4.0,2.0,8,NaN,3.0,NaN,7,4,...,18400.0,NaN,36400.0,NaN,33600.0,NaN,NaN,NaN,46200.0,53800.0
1,0,2,11.0,2.0,8,NaN,6.0,NaN,11,1,...,NaN,16000.0,NaN,10000.0,NaN,40000.0,NaN,NaN,20000.0,80000.0
2,0,3,3.0,1.0,7,NaN,3.0,NaN,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,28000.0,72000.0,28000.0,72000.0
3,1,3,3.0,1.0,8,NaN,3.0,NaN,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,50000.0,50000.0,50000.0
4,1,2,5.0,2.0,8,NaN,4.0,NaN,6,2,...,NaN,20000.0,NaN,50000.0,NaN,0.0,NaN,NaN,50000.0,50000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,61,2,4.0,2.0,8,NaN,4.0,NaN,7,4,...,NaN,15000.0,NaN,9000.0,NaN,41000.0,NaN,NaN,35000.0,65000.0
157,61,3,7.0,1.0,8,NaN,4.0,NaN,8,4,...,NaN,NaN,NaN,NaN,NaN,NaN,20000.0,80000.0,20000.0,80000.0
158,61,1,4.0,2.0,7,NaN,4.0,NaN,7,4,...,16800.0,NaN,13300.0,NaN,56700.0,NaN,NaN,NaN,23700.0,76300.0
159,62,2,8.0,1.0,4,NaN,4.0,NaN,5,5,...,NaN,8800.0,NaN,40000.0,NaN,10000.0,NaN,NaN,77000.0,23000.0


#### Number of valid cases

In [37]:
len(df_final)

161

In [38]:
df_final

,FS01_CP,FS01,SD03,SD05,SD11,SD11_10,SD14,SD14_08,SD16,MZ03_01,...,SAFE_ERSPART_GRUPPE_A,SAFE_ERSPART_GRUPPE_B,RISK_ERBE_GRUPPE_A,RISK_ERBE_GRUPPE_B,SAFE_ERBE_GRUPPE_A,SAFE_ERBE_GRUPPE_B,RISK_KONTROLLE,SAFE_KONTROLLE,TOTAL_RISIKO,TOTAL_SAFE
0,0,1,4.0,2.0,8,NaN,3.0,NaN,7,4,...,18400.0,NaN,36400.0,NaN,33600.0,NaN,NaN,NaN,46200.0,53800.0
1,0,2,11.0,2.0,8,NaN,6.0,NaN,11,1,...,NaN,16000.0,NaN,10000.0,NaN,40000.0,NaN,NaN,20000.0,80000.0
2,0,3,3.0,1.0,7,NaN,3.0,NaN,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,28000.0,72000.0,28000.0,72000.0
3,1,3,3.0,1.0,8,NaN,3.0,NaN,5,4,...,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,50000.0,50000.0,50000.0
4,1,2,5.0,2.0,8,NaN,4.0,NaN,6,2,...,NaN,20000.0,NaN,50000.0,NaN,0.0,NaN,NaN,50000.0,50000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,61,2,4.0,2.0,8,NaN,4.0,NaN,7,4,...,NaN,15000.0,NaN,9000.0,NaN,41000.0,NaN,NaN,35000.0,65000.0
157,61,3,7.0,1.0,8,NaN,4.0,NaN,8,4,...,NaN,NaN,NaN,NaN,NaN,NaN,20000.0,80000.0,20000.0,80000.0
158,61,1,4.0,2.0,7,NaN,4.0,NaN,7,4,...,16800.0,NaN,13300.0,NaN,56700.0,NaN,NaN,NaN,23700.0,76300.0
159,62,2,8.0,1.0,4,NaN,4.0,NaN,5,5,...,NaN,8800.0,NaN,40000.0,NaN,10000.0,NaN,NaN,77000.0,23000.0


### Save to excel file

In [39]:
df_final.to_excel("data_masterstudie_filtered.xlsx")